# Extract Data from TMDB

**Objective - Financial Data**
- Stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)
- As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001
- Each year should be saved as a separate .csv.gz file
- Create two functions for extracting data from TMDB
- In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.
- Save the final results to 2 separate .csv.gz files



Source: The Movie Database (TMDB)  https://www.themoviedb.org/

# Imports

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [3]:
import json
with open('C:/Users/david/OneDrive/Documents/May Files/.secret/tmdb.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['APIKey', 'APIReadAccessToken'])

In [4]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['APIKey']

# Querying Movies by ID

In [5]:
# make folder for API call data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [6]:
# function to add certification to movie.info dict
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    
    # set movie id
    movie = tmdb.Movies(movie_id)

    # save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # only get releases of the movie in the US
    for c in releases['countries']:
        
        # if country abbreviation == US
        if c['iso_3166_1'] == 'US':
            
            # save certification key into info dict
            info['certification'] = c['certification']
            
    return info

In [7]:
# test get_certification function
movie_id = 'tt0332280'
info = get_movie_with_rating(movie_id)
info['certification']

'PG-13'

In [8]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file
    (filename). Adapted from: https://www.geeksforgeeks.org/
    append-to-json-file-using-python/"""
    
    with open(filename, 'r+') as file:
        
        # load existing data from file into dict
        file_data = json.load(file)
        
        # if both types of data are lists, extend
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
            
        # else, append
        else:
            file_data.append(new_data)
            
        # set current position at offset
        file.seek(0)
        
        # convert to json
        json.dump(file_data, file)

In [9]:
# load basics from Data folder
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [10]:
# define years requested
YEARS_TO_GET = [2000, 2001]

# create empty list to save errors
errors = []

In [11]:
# set up progress bar for outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):
    
    # select JSON_FILE name to save results in progress
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # check if file already exists
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        
        # save an empty dict with "imdb_id" = 0 to new file
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}], f)
    
    # define df for current year
    df = basics.loc[basics['startYear'] == YEAR].copy()
    
    # save movie ids from df['tconst'] to list
    movie_ids = df['tconst'].copy()
    
    # load any existing data from json into previous_df
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any movies that already exist in JSON_FILE
    # (so as not to repeat any API calls for the same movie)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # inner loop to make API calls
    # iterate through list of movie_ids_to_get
    # some movies with imdb_ids don't exist in tmdb, so use
    # try/except to work through errors
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc = f'Movies from {YEAR}',
                                 position = 1,
                                 leave = True):
        
        try:
            # retrieve data for movie_id
            temp = get_movie_with_rating(movie_id)
            
            # append/extend results to JSON_FILE
            write_json(temp, JSON_FILE)
            
            # pause to prevent overwhelming server with calls
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    # save year's results in csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                        compression = "gzip",
                        index = False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1456 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1575 [00:00<?, ?it/s]

In [12]:
# print number of errors encountered
print(f"- Total errors: {len(errors)}")

- Total errors: 441
